In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import sqlite3
from sqlite3 import Error
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from pulp import *
import re
import scipy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, r2_score
from sklearn.model_selection import cross_val_score
from sklearn import svm
from mealpy.swarm_based.MFO import BaseMFO
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [2]:
season16Dir = "data/2016-17/gws/gw"
season17Dir = "data/2017-18/gws/gw"
season18Dir = "data/2018-19/gws/gw"
season19Dir = "data/2019-20/gws/gw"

currentSeasonDir = "data/2020-21/gws/gw"
CURRENT_GW = 9

In [3]:
season16 = pd.read_csv(season16Dir+"1.csv", engine="python")
season17 = pd.read_csv(season17Dir+"1.csv", engine="python")
season18 = pd.read_csv(season18Dir+"1.csv", engine="python")
# season16 = pd.read_csv(season19Dir+"1.csv", engine="python")

# season16GWs = []
# season17GWs = []
# season18GWs = []
# season19GWs = []
for i in range(2,38):
    season16 = season16.append(pd.read_csv(season16Dir+str(i)+".csv", engine='python'))
    season17 = season17.append(pd.read_csv(season17Dir+str(i)+".csv", engine='python'))
    season18 = season18.append(pd.read_csv(season18Dir+str(i)+".csv", engine='python'))
#     season19.append(pd.read_csv(season19Dir+str(i)+".csv", engine='python'))
print(season16)
season16 = season16[["assists","bonus","bps",
                   "clean_sheets","goals_conceded","goals_scored","minutes","red_cards",
                    "round", "saves","total_points",
                   "yellow_cards","name"]]
season17 = season17[["assists","bonus","bps",
                   "clean_sheets","goals_conceded","goals_scored","minutes","red_cards",
                    "round", "saves","total_points",
                   "yellow_cards","name"]]
season18 = season18[["assists","bonus","bps",
                   "clean_sheets","goals_conceded","goals_scored","minutes","red_cards",
                    "round", "saves","total_points",
                   "yellow_cards","name"]]
# season19 = season19["assists","attempted_passes","big_chances_created","bonus","bps",
#                    "clean_sheets","clearances_blocks_interceptions","completed_passes",
#                    "dribbles","errors_leading_to_goal","errors_leading_to_goal_attempt",
#                    "fouls","goals_conceded","goals_scored","key_passes","minutes","offside",
#                    "open_play_crosses","recoveries","red_cards","round","saves","selected",
#                    "tackled","tackles", "target_missed","total_points","transfers_in",
#                    "transfers_out","yellow_cards","name"]

                    name  assists  attempted_passes  big_chances_created  \
0        Aaron_Cresswell        0                 0                    0   
1           Aaron_Lennon        0                 3                    0   
2           Aaron_Ramsey        0                26                    0   
3     Abdoulaye_Doucour�        0                 0                    0   
4      Abdul Rahman_Baba        0                 0                    0   
...                  ...      ...               ...                  ...   
1003       Younes_Kaboul        0                 0                    0   
1004  Zlatan_Ibrahimovic        0                 0                    0   
1005  Zlatan_Ibrahimovic        0                 0                    0   
1006      �lvaro_Arbeloa        0                 0                    0   
1007      �lvaro_Negredo        0                23                    0   

      big_chances_missed  bonus  bps  clean_sheets  \
0                      0      0  

In [11]:
completeGws = []
for i in range(5,38):
    w1 = season16.loc[season16["round"]==i-4]
    w2 = season16.loc[season16["round"]==i-3]
    w3 = season16.loc[season16["round"]==i-2]
    w4 = season16.loc[season16["round"]==i-1]
    w5Points = season16.loc[season16["round"]==i][["total_points","name"]]
    temp16 = pd.merge(w1,w2,on="name",suffixes=["","gw2"],how="inner")
    temp16 = pd.merge(temp16,w3,on="name",suffixes=["","gw3"],how="inner")
    temp16 = pd.merge(temp16,w4,on="name",suffixes=["","gw4"],how="inner")
    temp16 = pd.merge(temp16,w5Points,on="name",suffixes=["","gw5"],how="inner")
    
    w1 = season17.loc[season17["round"]==i-4]
    w2 = season17.loc[season17["round"]==i-3]
    w3 = season17.loc[season17["round"]==i-2]
    w4 = season17.loc[season17["round"]==i-1]
    w5Points = season17.loc[season17["round"]==i][["total_points","name"]]
    temp17 = pd.merge(w1,w2,on="name",suffixes=["","gw2"],how="inner")
    temp17 = pd.merge(temp17,w3,on="name",suffixes=["","gw3"],how="inner")
    temp17 = pd.merge(temp17,w4,on="name",suffixes=["","gw4"],how="inner")
    temp17 = pd.merge(temp17,w5Points,on="name",suffixes=["","gw5"],how="inner")
    
    w1 = season18.loc[season18["round"]==i-4]
    w2 = season18.loc[season18["round"]==i-3]
    w3 = season18.loc[season18["round"]==i-2]
    w4 = season18.loc[season18["round"]==i-1]
    w5Points = season18.loc[season18["round"]==i][["total_points","name"]]
    temp18 = pd.merge(w1,w2,on="name",suffixes=["","gw2"],how="inner")
    temp18 = pd.merge(temp18,w3,on="name",suffixes=["","gw3"],how="inner")
    temp18 = pd.merge(temp18,w4,on="name",suffixes=["","gw4"],how="inner")
    temp18 = pd.merge(temp18,w5Points,on="name",suffixes=["","gw5"],how="inner")
    
    completeGws.append(temp16)
    completeGws.append(temp17)
    completeGws.append(temp18)

In [12]:
completeGws[0]

,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,minutes,red_cards,round,saves,...,clean_sheetsgw4,goals_concededgw4,goals_scoredgw4,minutesgw4,red_cardsgw4,roundgw4,savesgw4,total_pointsgw4,yellow_cardsgw4,total_pointsgw5
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,4,0,0,0,0
1,0,0,6,0,0,0,15,0,1,0,...,0,0,0,0,0,4,0,0,0,0
2,0,0,5,0,3,0,60,0,1,0,...,0,0,0,0,0,4,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,4,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,4,0,0,0,0
520,0,0,6,0,0,0,14,0,1,0,...,0,0,0,0,0,4,0,0,0,0
521,0,0,18,0,2,0,90,0,1,0,...,0,2,0,81,0,4,0,1,0,0
522,0,3,35,0,1,1,90,0,1,0,...,0,2,1,90,0,4,0,5,1,5


In [13]:
df = pd.DataFrame(completeGws[0])

In [14]:
for i in range(1,len(completeGws)):
    df=df.append(completeGws[i])

In [15]:
df = df.drop(columns=["name","round","roundgw2","roundgw3","roundgw4"])
x = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [16]:
md = RandomForestRegressor(bootstrap=True, criterion="mse", max_depth=10, n_estimators=1000,
                          max_features ="log2", min_samples_leaf=3, min_samples_split=5, n_jobs=-1, verbose=1 )
md.fit(x, y)
# prediction = cross_val_score(md, x, y, cv=10, scoring="r2")
# print(prediction)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.2s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=1, warm_start=False)

In [16]:
space = {'criterion' : hp.choice("criterion",['mse']),
         'max_depth' : hp.choice("depth",[10,25,50,100]),
         'min_samples_split' : hp.choice("minSamplesSplit",[2,3,4,5,6,7,8,9,10,11,12]),
         'min_samples_leaf' : hp.choice("minSamplesLeaf",[1,2,3,4,5,6,7,8,9,10]),
         'max_features' : hp.choice("maxFeat",["auto","log2",None]),
         'bootstrap':hp.choice("bootstrap",[True,False]),
         'n_estimators':hp.choice("estimators",[100,250,500])}

def objective(space):
    print(space)
    md = RandomForestRegressor(criterion=space['criterion'],
                                max_depth=space['max_depth'],
                                min_samples_split=space["min_samples_split"],
                                min_samples_leaf=space["min_samples_leaf"],
                                max_features=space["max_features"],
                                bootstrap=space["bootstrap"],
                                n_estimators=space["n_estimators"],
                                n_jobs=-1, verbose=1)
    accuracy = cross_val_score(md,x,y, cv=5, scoring="r2").mean()
    return{'loss': -accuracy,'status':STATUS_OK}

best = fmin(fn=objective,
            space=space,
            algo= tpe.suggest,
            max_evals = 100,
            trials = Trials())

best

{'bootstrap': True, 'criterion': 'mse', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 250}
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.1s



  0%|          | 0/100 [00:03<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [23]:
def MFOObjectiveFuncRF(xPassed, alpha=0.05):
    colLen = len(x.columns)
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
#     print(x_test)
    md = RandomForestRegressor(criterion="mse",n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=10, min_samples_split=11, n_jobs = -1, verbose=0)
    md.fit(x_test,y)
    performance = cross_val_score(md, x, y, cv=5, scoring="r2").mean()

    result = -performance
    return result


In [ ]:
md =BaseMFO(MFOObjectiveFuncRF,33,(0,1),40, 40)
pos, fit, loss = md._train__()
print(fit)          
# performance = md.predict(x_test)
# performance = accuracy_score(y, performance)
print(pos)    
mask = np.reshape(pos/2,(1,-1))
mask = np.round(mask)
mask = mask[0]
print(mask)

> Epoch: 1, Best fit: -0.25500313282565557
> Epoch: 2, Best fit: -0.25500313282565557
> Epoch: 3, Best fit: -0.25500313282565557
> Epoch: 4, Best fit: -0.25500313282565557
> Epoch: 5, Best fit: -0.25500313282565557
> Epoch: 6, Best fit: -0.25500313282565557
> Epoch: 7, Best fit: -0.25500313282565557
> Epoch: 8, Best fit: -0.25500313282565557
> Epoch: 9, Best fit: -0.25500313282565557
> Epoch: 10, Best fit: -0.25500313282565557
> Epoch: 11, Best fit: -0.25500313282565557
> Epoch: 12, Best fit: -0.25500313282565557
> Epoch: 13, Best fit: -0.25500313282565557
> Epoch: 14, Best fit: -0.25500313282565557
> Epoch: 15, Best fit: -0.25500313282565557
> Epoch: 16, Best fit: -0.2550243866140554
> Epoch: 17, Best fit: -0.2552267338968589


In [10]:
for item in pos:
    if item>0:
        mask.append(1)
    else:
        mask.append(0)
print(mask)
xConv = x.to_numpy()
print(len(xConv[0]))
removals = []
for i in range(0,len(mask)-1):
    if mask[i] == 0:
        removals.append(i+1)

print(removals)
for removal in reversed(removals):
    xConv = np.delete(xConv,removal,1)
xFeatures = pd.DataFrame(data=xConv)
md = RandomForestRegressor(criterion="mse",n_estimators = 1000,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=10, min_samples_split=11, n_jobs = -1, verbose=1)
md.fit(xFeatures,y)
prediction = cross_val_score(md, xFeatures, y, cv=10, scoring="r2")
print(prediction)

# xConv = lastSeason.to_numpy()
# xFeatures = xConv[:,mask==1]
# prediction = md.predict(xFeatures)
# print(prediction.round())
# print(len(prediction))
# print(len(xFeatures))

NameError: name 'pos' is not defined

In [17]:
currentSeason =[]
currentSeason = pd.read_csv(currentSeasonDir+str(CURRENT_GW-4)+".csv", engine="python")
# print(currentSeason)

# for i in range(CURRENT_GW-2,CURRENT_GW):
#     currentSeason = currentSeason.append(pd.read_csv(currentSeasonDir+str(i)+".csv", engine='python'))
currentSeason = currentSeason.append(pd.read_csv(currentSeasonDir+str(CURRENT_GW-3)+".csv", engine='python'))
currentSeason = currentSeason.append(pd.read_csv(currentSeasonDir+str(CURRENT_GW-2)+".csv", engine='python'))
currentSeason = currentSeason.append(pd.read_csv(currentSeasonDir+str(CURRENT_GW-1)+".csv", engine='python'))

w1 = currentSeason.loc[currentSeason["round"]==CURRENT_GW-4]
w2 = currentSeason.loc[currentSeason["round"]==CURRENT_GW-3]
w3 = currentSeason.loc[currentSeason["round"]==CURRENT_GW-2]
w4 = currentSeason.loc[currentSeason["round"]==CURRENT_GW-1]
temp = pd.merge(w1,w2,on="name",suffixes=["","gw2"],how="inner")
temp = pd.merge(temp,w3,on="name",suffixes=["","gw3"],how="inner")
temp = pd.merge(temp,w4,on="name",suffixes=["","gw4"],how="inner")
df = temp
# print(df.columns)
df2 = df[["assists","bonus","bps",
                   "clean_sheets","goals_conceded","goals_scored","minutes","red_cards",
                     "saves","total_points","yellow_cards",
        'assistsgw2', 'bonusgw2', 'bpsgw2',
       'clean_sheetsgw2',
       'goals_concededgw2', 'goals_scoredgw2', 'minutesgw2','red_cardsgw2',
       'savesgw2', 'total_pointsgw2',
       'yellow_cardsgw2', 
        'assistsgw3', 'bonusgw3', 'bpsgw3',
       'clean_sheetsgw3',
       'goals_concededgw3', 'goals_scoredgw3','minutesgw3', 'red_cardsgw3',
       'savesgw3', 'total_pointsgw3',
       'yellow_cardsgw3',
                 'assistsgw4', 'bonusgw4', 'bpsgw4',
       'clean_sheetsgw4',
       'goals_concededgw4', 'goals_scoredgw4','minutesgw4', 'red_cardsgw4',
       'savesgw4', 'total_pointsgw4',
       'yellow_cardsgw4']]
# df2 = df2.drop(columns="name")
x = df2
print(x)
prediction = md.predict(x)
df["prediction"] = prediction
df.to_csv("gw_prediction.csv")

     assists  bonus  bps  clean_sheets  goals_conceded  goals_scored  minutes  \
0          0      0    2             1               0             0       67   
1          0      0   15             0               1             0       90   
2          0      0    0             0               0             0        0   
3          0      0    0             0               0             0        0   
4          0      0    0             0               0             0        0   
..       ...    ...  ...           ...             ...           ...      ...   
578        0      0    0             0               0             0        0   
579        0      1   28             0               2             1       90   
580        1      0   25             0               3             0       90   
581        0      0    0             0               0             0        0   
582        0      0    0             0               0             0        0   

     red_cards  saves  tota

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [4]:
import numpy as np
import pandas as pd
import pulp
import os
import argparse
import warnings


#needs use_now_cost
#start_cost
#position
#team_id
#full_name
#prediction
def add_position_dummy(df):
    for p in df.position.unique():
        df['is_' + str(p).lower()] = np.where(df.position == p, int(1), int(0))
    return df


def add_team_dummy(df):
    for t in df.team.unique():
        df['team_' + str(t).lower()] = np.where(df.team == t, int(1), int(0))
    return df


def get_optimal_squad(data, formation='2-5-5-3', budget=100.0, season='2019/20',
                      use_now_cost=True, optimise_on='prediction'):
    min_player_cost = 4.
    n_players = sum(int(i) for i in formation.split('-'))
    max_budget = 100 - (15 - n_players)*min_player_cost

    if budget is None:
        budget = max_budget

    if budget > max_budget:
        warnings.warn('Supplied budget exceeds expected maximum of '
                      '{0}'.format(max_budget))

    season_stats = (
        data
        .reset_index()
        .assign(cost=lambda df: (df.value / 10.) if use_now_cost else (df.start_cost / 10.))
        .pipe(add_position_dummy)
        .pipe(add_team_dummy)
    )

    players = season_stats.name

    # initalise the problem
    fpl_problem = pulp.LpProblem('FPL', pulp.LpMaximize)

    # create a dictionary of pulp variables with keys from names
    x = pulp.LpVariable.dict('x_ % s', players, lowBound=0, upBound=1,
                             cat=pulp.LpInteger)

    # player score data
    player_points = dict(
        zip(season_stats.name, np.array(season_stats[optimise_on])))

    # objective function
    fpl_problem += sum([player_points[i] * x[i] for i in players])

    # constraints
    position_names = ['gk', 'def', 'mid', 'fwd']
    position_constraints = [int(i) for i in formation.split('-')]

    constraints = dict(zip(position_names, position_constraints))
    constraints['total_cost'] = budget
    constraints['team'] = 3

    # could get straight from dataframe...
    player_cost = dict(zip(season_stats.name, season_stats.cost))
    player_position = dict(zip(season_stats.name, season_stats.position))
    player_gk = dict(zip(season_stats.name, season_stats.is_goalkeeper))
    player_def = dict(zip(season_stats.name, season_stats.is_defender))
    player_mid = dict(zip(season_stats.name, season_stats.is_midfielder))
    player_fwd = dict(zip(season_stats.name, season_stats.is_forward))

    # apply the constraints
    fpl_problem += sum([player_cost[i] * x[i] for i in players]) <= float(constraints['total_cost'])
    fpl_problem += sum([player_gk[i] * x[i] for i in players]) == constraints['gk']
    fpl_problem += sum([player_def[i] * x[i] for i in players]) == constraints['def']
    fpl_problem += sum([player_mid[i] * x[i] for i in players]) == constraints['mid']
    fpl_problem += sum([player_fwd[i] * x[i] for i in players]) == constraints['fwd']

    for t in season_stats.team:
        player_team = dict(
            zip(season_stats.name, season_stats['team_' + str(t)]))
        fpl_problem += sum([player_team[i] * x[i] for i in players]) <= constraints['team']

    # solve the thing
    fpl_problem.solve()

    total_points = 0.
    total_cost = 0.
    optimal_squad = []
    for p in players:
        if x[p].value() != 0:
            total_points += player_points[p]
            total_cost += player_cost[p]

            optimal_squad.append({
                'name': p,
                'position': player_position[p],
                'cost': player_cost[p],
                'points': player_points[p]
            })

    solution_info = {
        'formation': formation,
        'total_points': total_points,
        'total_cost': total_cost
    }

    return optimal_squad, solution_info


In [5]:
# df2 = season1920
df2 = pd.read_csv("gw_prediction.csv")
print(df2)
positions_csv = pd.read_csv("data/2020-21/players_raw.csv", engine='python')
positions_csv = positions_csv[["first_name","second_name","chance_of_playing_next_round",
                              "element_type","now_cost","team"]]
positions_csv["nameFixed"] = positions_csv["first_name"]+" "+positions_csv["second_name"]
# df2["nameFixed"] = df2["first_name"]+" "+df2["second_name"]
stuffs = []
for name in df2['name']:
    stuffs.append(name.split("_"))
# print(stuffs)
for idx,name in enumerate(stuffs):
    name = name[:-1]
    part = ""
    for parts in name:
        part += parts
        part += " "
    stuffs[idx] =  part.rstrip()
#add stuffs, chance of playing, positions

df2["nameFixed"] = stuffs
# print(df2["nameFixed"])
# print(positions_csv["nameFixed"])
df2 = pd.merge(df2,positions_csv,on="nameFixed",suffixes=["new","old"],how="inner")
print(df2.columns)
df2["position"] = df2["element_type"]
df2["is_goalkeeper"] = np.where(df2["position"]==1,1,0)
df2["is_defender"] = np.where(df2["position"]==2,1,0)
df2["is_midfielder"] = np.where(df2["position"]==3,1,0)
df2["is_forward"] = np.where(df2["position"]==4,1,0)
# results = df2["chance_of_playing_next_round"]=="100"
# df2 = df2[results]
# print(results)
squad, soln = get_optimal_squad(df2,formation='1-3-4-3', budget=84.0)
print(squad)
squad = pd.DataFrame(squad)
squad.columns = [col.capitalize() for col in squad.columns]

# Display output
print(squad.to_string(index=False))
print('')  # extra line break
for key, val in soln.items():
    pretty_key = key.replace('_', ' ').capitalize()
    print('{0}:\t{1}'.format(pretty_key, val))

     Unnamed: 0                       name  assists  bonus  bps  clean_sheets  \
0             0           Dennis_Praet_225        0      1   29             0   
1             1         Oliver_Norwood_349        0      0    0             0   
2             2       Giovani_Lo Celso_403        0      0    2             0   
3             3  Alisson_Ramses Becker_252        0      2   41             1   
4             4           Paul_Dummett_331        0      0    0             0   
..          ...                        ...      ...    ...  ...           ...   
534         534      Marcus_Bettinelli_183        0      0    0             0   
535         535          Michael_Keane_156        0      0   28             0   
536         536      Benjamin_Chilwell_232        0      0    0             0   
537         537            Jordan_Ayew_140        1      0   13             0   
538         538       Stuart_Armstrong_368        0      0   11             0   

     creativity  element  f

NameError: name 'optimalTeam' is not defined